# Enable Agent Tutorial Part 2: Enable Agent 구현

## 개요

Part 1에서 학습한 PyTorch 딥러닝 모델을 Enable Agent로 변환한다. Enable Agent는 기존 ML/DL 모델을 LLM 기반 시스템에 통합하기 위한 래퍼 패턴이다.


## 학습 내용

- YAML 기반 스킬 정의 파일 작성
- Enable Agent 클래스 구현
- OpenAI Function Calling 통합
- AI 기반 예측 설명 생성

---
## 1. 라이브러리 임포트

In [2]:
import os
import json
import yaml
import joblib
import numpy as np
from pathlib import Path
from datetime import datetime
from typing import Dict, Any, Tuple, List

import torch
import torch.nn as nn

from dotenv import load_dotenv
from openai import OpenAI

In [3]:
# .env 파일에서 환경 변수를 로드한다
!echo "OPENAI_API_KEY=sk-proj-OpenAI API Key 붙혀넣기" >> .env

load_dotenv()

True

---
## 2. YAML 스킬 파일 생성

스킬 파일은 Enable Agent의 명세를 정의한다. 이 파일에는 에이전트의 기능, 입출력 스키마, 모델 정보가 포함된다.

In [4]:
skill_definition = {
    'agent_name': 'SpaceshipTransportPredictor',
    'version': '1.0.0',
    'description': 'Spaceship Titanic 승객의 다른 차원 이동 여부를 예측하는 딥러닝 에이전트',
    
    'capabilities': {
        'primary': '승객 정보를 기반으로 다른 차원 이동(Transported) 여부를 예측한다',
        'secondary': [
            '예측 확률 및 신뢰도 제공',
            '입력 데이터 유효성 검증',
            'AI 기반 예측 설명 생성'
        ]
    },
    
    'input_schema': {
        'type': 'object',
        'properties': {
            'HomePlanet': {
                'type': 'integer',
                'description': '출발 행성 (0: Earth, 1: Europa, 2: Mars, 3: Unknown)',
                'minimum': 0,
                'maximum': 3
            },
            'CryoSleep': {
                'type': 'integer',
                'description': '냉동 수면 여부 (0: No, 1: Yes)',
                'minimum': 0,
                'maximum': 1
            },
            'Destination': {
                'type': 'integer',
                'description': '목적지 (0: TRAPPIST-1e, 1: PSO J318.5-22, 2: 55 Cancri e)',
                'minimum': 0,
                'maximum': 2
            },
            'Age': {
                'type': 'number',
                'description': '나이',
                'minimum': 0,
                'maximum': 100
            },
            'VIP': {
                'type': 'integer',
                'description': 'VIP 서비스 여부 (0: No, 1: Yes)',
                'minimum': 0,
                'maximum': 1
            },
            'RoomService': {
                'type': 'number',
                'description': '룸서비스 지출액',
                'minimum': 0
            },
            'FoodCourt': {
                'type': 'number',
                'description': '푸드코트 지출액',
                'minimum': 0
            },
            'ShoppingMall': {
                'type': 'number',
                'description': '쇼핑몰 지출액',
                'minimum': 0
            },
            'Spa': {
                'type': 'number',
                'description': '스파 지출액',
                'minimum': 0
            },
            'VRDeck': {
                'type': 'number',
                'description': 'VR 데크 지출액',
                'minimum': 0
            }
        },
        'required': ['HomePlanet', 'CryoSleep', 'Destination', 'Age', 'VIP', 
                     'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
    },
    
    'output_schema': {
        'type': 'object',
        'properties': {
            'predicted_class': {'type': 'string', 'description': '예측된 이동 여부'},
            'probability': {'type': 'number', 'description': '이동될 확률 (0-1)'},
            'confidence': {'type': 'number', 'description': '예측 신뢰도 (0-1)'},
            'input_features': {'type': 'object', 'description': '입력된 특성 값'},
            'timestamp': {'type': 'string', 'description': '예측 수행 시각'}
        }
    },
    
    'model_info': {
        'model_path': 'models/spaceship_classifier.pth',
        'scaler_path': 'models/spaceship_scaler.pkl',
        'metadata_path': 'models/spaceship_classifier_metadata.json',
        'framework': 'PyTorch'
    },
    
    'encoding_maps': {
        'HomePlanet': {0: 'Earth', 1: 'Europa', 2: 'Mars', 3: 'Unknown'},
        'Destination': {0: 'TRAPPIST-1e', 1: 'PSO J318.5-22', 2: '55 Cancri e'},
        'CryoSleep': {0: 'No', 1: 'Yes'},
        'VIP': {0: 'No', 1: 'Yes'}
    }
}

Path('skills').mkdir(exist_ok=True)
skill_path = 'skills/spaceship_agent_skill.yaml'

with open(skill_path, 'w', encoding='utf-8') as f:
    yaml.dump(skill_definition, f, default_flow_style=False, allow_unicode=True, sort_keys=False)

print(f"스킬 파일 생성 완료: {skill_path}")

스킬 파일 생성 완료: skills/spaceship_agent_skill.yaml


In [5]:
with open(skill_path, 'r', encoding='utf-8') as f:
    print(f.read())

agent_name: SpaceshipTransportPredictor
version: 1.0.0
description: Spaceship Titanic 승객의 다른 차원 이동 여부를 예측하는 딥러닝 에이전트
capabilities:
  primary: 승객 정보를 기반으로 다른 차원 이동(Transported) 여부를 예측한다
  secondary:
  - 예측 확률 및 신뢰도 제공
  - 입력 데이터 유효성 검증
  - AI 기반 예측 설명 생성
input_schema:
  type: object
  properties:
    HomePlanet:
      type: integer
      description: '출발 행성 (0: Earth, 1: Europa, 2: Mars, 3: Unknown)'
      minimum: 0
      maximum: 3
    CryoSleep:
      type: integer
      description: '냉동 수면 여부 (0: No, 1: Yes)'
      minimum: 0
      maximum: 1
    Destination:
      type: integer
      description: '목적지 (0: TRAPPIST-1e, 1: PSO J318.5-22, 2: 55 Cancri e)'
      minimum: 0
      maximum: 2
    Age:
      type: number
      description: 나이
      minimum: 0
      maximum: 100
    VIP:
      type: integer
      description: 'VIP 서비스 여부 (0: No, 1: Yes)'
      minimum: 0
      maximum: 1
    RoomService:
      type: number
      description: 룸서비스 지출액
      minimum: 0
    FoodCourt:
      ty

---
## 3. PyTorch 모델 클래스 정의

Part 1에서 정의한 것과 동일한 모델 클래스를 정의한다. 모델 로드를 위해 필요하다.

In [6]:
class SpaceshipClassifier(nn.Module):
    """Spaceship Titanic 이진 분류 신경망"""
    
    def __init__(self, input_dim=10, hidden_dims=[64, 32, 16], dropout_rate=0.3):
        super(SpaceshipClassifier, self).__init__()
        
        layers = []
        prev_dim = input_dim
        
        for i, hidden_dim in enumerate(hidden_dims):
            layers.append(nn.Linear(prev_dim, hidden_dim))
            layers.append(nn.ReLU())
            if i < len(hidden_dims) - 1:
                layers.append(nn.Dropout(dropout_rate))
            prev_dim = hidden_dim
        
        layers.append(nn.Linear(prev_dim, 1))
        layers.append(nn.Sigmoid())
        
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x).squeeze()

print("SpaceshipClassifier 클래스 정의 완료")

SpaceshipClassifier 클래스 정의 완료


---
## 4. SpaceshipEnableAgent 클래스 구현

### 주요 기능

- `validate_input()`: 입력 데이터 유효성 검증
- `predict()`: 모델 예측 실행
- `generate_explanation()`: AI 기반 예측 설명 생성
- `generate_tool_definition()`: OpenAI Function Calling 도구 정의 생성
- `get_capability_description()`: 에이전트 기능 설명 반환

In [7]:
class SpaceshipEnableAgent:
    """Spaceship Titanic 예측 모델을 Enable Agent로 래핑한 클래스"""
    
    def __init__(self, skill_path: str):
        """Enable Agent 초기화"""
        with open(skill_path, 'r', encoding='utf-8') as f:
            self.skill = yaml.safe_load(f)
        
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
        # 모델 로드
        model_path = self.skill['model_info']['model_path']
        checkpoint = torch.load(model_path, map_location=self.device, weights_only=True)
        
        self.model = SpaceshipClassifier(
            input_dim=checkpoint['input_dim'],
            hidden_dims=checkpoint['hidden_dims'],
            dropout_rate=checkpoint['dropout_rate']
        ).to(self.device)
        self.model.load_state_dict(checkpoint['model_state_dict'])
        self.model.eval()
        
        # 스케일러 로드
        scaler_path = self.skill['model_info']['scaler_path']
        self.scaler = joblib.load(scaler_path)
        
        # 메타데이터 로드
        metadata_path = self.skill['model_info']['metadata_path']
        with open(metadata_path, 'r', encoding='utf-8') as f:
            self.metadata = json.load(f)
        
        # OpenAI 클라이언트
        self.client = OpenAI()
        
        # 인코딩 맵
        self.encoding_maps = self.skill['encoding_maps']
        
        print(f"Enable Agent 초기화 완료: {self.skill['agent_name']}")
        print(f"모델 정확도: {self.metadata['test_accuracy']:.4f}")
        print(f"디바이스: {self.device}")
    
    def get_capability_description(self) -> str:
        """에이전트의 기능을 자연어로 반환한다"""
        return f"""
Agent 이름: {self.skill['agent_name']}
버전: {self.skill['version']}
설명: {self.skill['description']}

주요 기능: {self.skill['capabilities']['primary']}
부가 기능: {', '.join(self.skill['capabilities']['secondary'])}

입력 파라미터:
- HomePlanet: 출발 행성 (0: Earth, 1: Europa, 2: Mars, 3: Unknown)
- CryoSleep: 냉동 수면 여부 (0: No, 1: Yes)
- Destination: 목적지 (0: TRAPPIST-1e, 1: PSO J318.5-22, 2: 55 Cancri e)
- Age: 나이
- VIP: VIP 서비스 여부 (0: No, 1: Yes)
- RoomService: 룸서비스 지출액
- FoodCourt: 푸드코트 지출액
- ShoppingMall: 쇼핑몰 지출액
- Spa: 스파 지출액
- VRDeck: VR 데크 지출액

출력:
- predicted_class: 예측된 이동 여부 (Transported / Not Transported)
- probability: 이동될 확률 (0-1)
- confidence: 예측 신뢰도 (0-1)

모델 정보:
- 타입: {self.metadata['model_type']}
- 아키텍처: {self.metadata['architecture']}
- 테스트 정확도: {self.metadata['test_accuracy']:.4f}
- AUC-ROC: {self.metadata['test_auc_roc']:.4f}
""".strip()
    
    def validate_input(self, input_data: Dict[str, Any]) -> Tuple[bool, str]:
        """입력 데이터의 유효성을 검증한다"""
        required_fields = self.skill['input_schema']['required']
        properties = self.skill['input_schema']['properties']
        
        for field in required_fields:
            if field not in input_data:
                return False, f"필수 필드 누락: {field}"
        
        for field, value in input_data.items():
            if field in properties:
                prop = properties[field]
                if 'minimum' in prop and value < prop['minimum']:
                    return False, f"{field}가 최소값({prop['minimum']})보다 작다: {value}"
                if 'maximum' in prop and value > prop['maximum']:
                    return False, f"{field}가 최대값({prop['maximum']})보다 크다: {value}"
        
        return True, "유효한 입력"
    
    def _decode_categorical(self, input_data: Dict[str, Any]) -> Dict[str, str]:
        """범주형 변수를 사람이 읽을 수 있는 형태로 변환한다"""
        decoded = {}
        for field, value in input_data.items():
            if field in self.encoding_maps:
                decoded[field] = self.encoding_maps[field].get(value, str(value))
            else:
                decoded[field] = value
        return decoded
    
    def predict(self, input_data: Dict[str, Any]) -> Dict[str, Any]:
        """예측을 수행한다"""
        is_valid, message = self.validate_input(input_data)
        if not is_valid:
            raise ValueError(message)
        
        feature_names = self.metadata['feature_names']
        input_array = np.array([[input_data[f] for f in feature_names]])
        
        input_scaled = self.scaler.transform(input_array)
        input_tensor = torch.FloatTensor(input_scaled).to(self.device)
        
        with torch.no_grad():
            probability = self.model(input_tensor).item()
        
        predicted_class = 'Transported' if probability > 0.5 else 'Not Transported'
        confidence = probability if probability > 0.5 else (1 - probability)
        
        decoded_features = self._decode_categorical(input_data)
        
        return {
            "predicted_class": predicted_class,
            "probability": float(probability),
            "confidence": float(confidence),
            "input_features": input_data,
            "decoded_features": decoded_features,
            "timestamp": datetime.now().isoformat()
        }
    
    def generate_explanation(self, prediction_result: Dict[str, Any]) -> str:
        """예측 결과에 대한 AI 설명을 생성한다"""
        decoded = prediction_result['decoded_features']
        
        prompt = f"""
Spaceship Titanic 승객의 다른 차원 이동 예측 결과를 분석하고 설명해달라.

## 승객 정보
- 출발 행성: {decoded['HomePlanet']}
- 냉동 수면: {decoded['CryoSleep']}
- 목적지: {decoded['Destination']}
- 나이: {decoded['Age']}세
- VIP: {decoded['VIP']}
- 룸서비스 지출: {decoded['RoomService']}
- 푸드코트 지출: {decoded['FoodCourt']}
- 쇼핑몰 지출: {decoded['ShoppingMall']}
- 스파 지출: {decoded['Spa']}
- VR 데크 지출: {decoded['VRDeck']}

## 예측 결과
- 예측: {prediction_result['predicted_class']}
- 이동 확률: {prediction_result['probability']:.2%}
- 신뢰도: {prediction_result['confidence']:.2%}

이 예측 결과가 왜 나왔는지 승객의 특성을 바탕으로 2-3문장으로 간략히 설명해달라.
특히 냉동 수면 여부, 지출 패턴, 출발 행성이 예측에 어떤 영향을 미쳤는지 분석해달라.
문장의 끝은 ~다로 끝낸다.
"""
        
        response = self.client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7,
            max_tokens=300
        )
        
        return response.choices[0].message.content
    
    def generate_tool_definition(self) -> Dict[str, Any]:
        """OpenAI Function Calling을 위한 도구 정의를 생성한다"""
        return {
            "type": "function",
            "function": {
                "name": "predict_spaceship_transport",
                "description": self.skill['description'],
                "parameters": {
                    "type": "object",
                    "properties": {
                        "HomePlanet": {
                            "type": "integer",
                            "description": "출발 행성 (0: Earth, 1: Europa, 2: Mars, 3: Unknown)"
                        },
                        "CryoSleep": {
                            "type": "integer",
                            "description": "냉동 수면 여부 (0: No, 1: Yes)"
                        },
                        "Destination": {
                            "type": "integer",
                            "description": "목적지 (0: TRAPPIST-1e, 1: PSO J318.5-22, 2: 55 Cancri e)"
                        },
                        "Age": {
                            "type": "number",
                            "description": "나이"
                        },
                        "VIP": {
                            "type": "integer",
                            "description": "VIP 서비스 여부 (0: No, 1: Yes)"
                        },
                        "RoomService": {
                            "type": "number",
                            "description": "룸서비스 지출액"
                        },
                        "FoodCourt": {
                            "type": "number",
                            "description": "푸드코트 지출액"
                        },
                        "ShoppingMall": {
                            "type": "number",
                            "description": "쇼핑몰 지출액"
                        },
                        "Spa": {
                            "type": "number",
                            "description": "스파 지출액"
                        },
                        "VRDeck": {
                            "type": "number",
                            "description": "VR 데크 지출액"
                        }
                    },
                    "required": ["HomePlanet", "CryoSleep", "Destination", "Age", "VIP",
                                 "RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck"]
                }
            }
        }

print("SpaceshipEnableAgent 클래스 정의 완료")

SpaceshipEnableAgent 클래스 정의 완료


---
## 5. Enable Agent 초기화

In [8]:
agent = SpaceshipEnableAgent('skills/spaceship_agent_skill.yaml')

Enable Agent 초기화 완료: SpaceshipTransportPredictor
모델 정확도: 0.7947
디바이스: cpu


In [9]:
print(agent.get_capability_description())

Agent 이름: SpaceshipTransportPredictor
버전: 1.0.0
설명: Spaceship Titanic 승객의 다른 차원 이동 여부를 예측하는 딥러닝 에이전트

주요 기능: 승객 정보를 기반으로 다른 차원 이동(Transported) 여부를 예측한다
부가 기능: 예측 확률 및 신뢰도 제공, 입력 데이터 유효성 검증, AI 기반 예측 설명 생성

입력 파라미터:
- HomePlanet: 출발 행성 (0: Earth, 1: Europa, 2: Mars, 3: Unknown)
- CryoSleep: 냉동 수면 여부 (0: No, 1: Yes)
- Destination: 목적지 (0: TRAPPIST-1e, 1: PSO J318.5-22, 2: 55 Cancri e)
- Age: 나이
- VIP: VIP 서비스 여부 (0: No, 1: Yes)
- RoomService: 룸서비스 지출액
- FoodCourt: 푸드코트 지출액
- ShoppingMall: 쇼핑몰 지출액
- Spa: 스파 지출액
- VRDeck: VR 데크 지출액

출력:
- predicted_class: 예측된 이동 여부 (Transported / Not Transported)
- probability: 이동될 확률 (0-1)
- confidence: 예측 신뢰도 (0-1)

모델 정보:
- 타입: PyTorch Neural Network
- 아키텍처: SpaceshipClassifier
- 테스트 정확도: 0.7947
- AUC-ROC: 0.8772


---
## 6. 예측 테스트

In [10]:
# 냉동 수면 승객 (이동될 가능성 높음)
test_case_1 = {
    'HomePlanet': 0,      # Earth
    'CryoSleep': 1,       # Yes
    'Destination': 2,     # 55 Cancri e
    'Age': 28.0,
    'VIP': 0,             # No
    'RoomService': 0.0,
    'FoodCourt': 0.0,
    'ShoppingMall': 0.0,
    'Spa': 0.0,
    'VRDeck': 0.0
}

result_1 = agent.predict(test_case_1)

print("=== 테스트 케이스 1: 냉동 수면 승객 ===")
print(json.dumps(result_1, indent=2, ensure_ascii=False))

=== 테스트 케이스 1: 냉동 수면 승객 ===
{
  "predicted_class": "Transported",
  "probability": 0.6649807095527649,
  "confidence": 0.6649807095527649,
  "input_features": {
    "HomePlanet": 0,
    "CryoSleep": 1,
    "Destination": 2,
    "Age": 28.0,
    "VIP": 0,
    "RoomService": 0.0,
    "FoodCourt": 0.0,
    "ShoppingMall": 0.0,
    "Spa": 0.0,
    "VRDeck": 0.0
  },
  "decoded_features": {
    "HomePlanet": "Earth",
    "CryoSleep": "Yes",
    "Destination": "55 Cancri e",
    "Age": 28.0,
    "VIP": "No",
    "RoomService": 0.0,
    "FoodCourt": 0.0,
    "ShoppingMall": 0.0,
    "Spa": 0.0,
    "VRDeck": 0.0
  },
  "timestamp": "2025-12-16T18:13:47.326140"
}


In [11]:
# 고지출 승객 (이동 안될 가능성 높음)
test_case_2 = {
    'HomePlanet': 1,      # Europa
    'CryoSleep': 0,       # No
    'Destination': 0,     # TRAPPIST-1e
    'Age': 45.0,
    'VIP': 1,             # Yes
    'RoomService': 1500.0,
    'FoodCourt': 2000.0,
    'ShoppingMall': 500.0,
    'Spa': 3000.0,
    'VRDeck': 1000.0
}

result_2 = agent.predict(test_case_2)

print("=== 테스트 케이스 2: 고지출 VIP 승객 ===")
print(json.dumps(result_2, indent=2, ensure_ascii=False))

=== 테스트 케이스 2: 고지출 VIP 승객 ===
{
  "predicted_class": "Not Transported",
  "probability": 0.01183952298015356,
  "confidence": 0.9881604770198464,
  "input_features": {
    "HomePlanet": 1,
    "CryoSleep": 0,
    "Destination": 0,
    "Age": 45.0,
    "VIP": 1,
    "RoomService": 1500.0,
    "FoodCourt": 2000.0,
    "ShoppingMall": 500.0,
    "Spa": 3000.0,
    "VRDeck": 1000.0
  },
  "decoded_features": {
    "HomePlanet": "Europa",
    "CryoSleep": "No",
    "Destination": "TRAPPIST-1e",
    "Age": 45.0,
    "VIP": "Yes",
    "RoomService": 1500.0,
    "FoodCourt": 2000.0,
    "ShoppingMall": 500.0,
    "Spa": 3000.0,
    "VRDeck": 1000.0
  },
  "timestamp": "2025-12-16T18:13:48.228877"
}


---
## 7. AI 설명 생성 테스트

In [12]:
print("=== AI 설명: 테스트 케이스 1 ===")
explanation_1 = agent.generate_explanation(result_1)
print(explanation_1)

=== AI 설명: 테스트 케이스 1 ===
이 승객은 냉동 수면을 선택한 점이 긍정적으로 작용하여, 긴 여행 동안 생리적 스트레스를 줄이고 안정성을 높였을 가능성이 있다. 또한, 룸서비스와 같은 지출이 전혀 없다는 사실은 그가 여행 중 특별한 요구나 불만이 없음을 나타내며, 이는 이동 성공 확률을 높이는 요인으로 작용했을 것이다. 출발 행성이 Earth인 점은 우주 여행에 대한 익숙함과 안전성을 제공하여, 전체적인 이동 확률을 더 높였다고 볼 수 있다.


In [13]:
print("=== AI 설명: 테스트 케이스 2 ===")
explanation_2 = agent.generate_explanation(result_2)
print(explanation_2)

=== AI 설명: 테스트 케이스 2 ===
이 승객은 냉동 수면을 이용하지 않은 상태로, 이는 이동에 필요한 에너지를 절약하지 못하는 요인이 된다. 또한, 고액의 룸서비스와 스파 지출이 있지만 상대적으로 낮은 이동 확률을 보이는 것은 출발 행성인 Europa의 특성과 맞물려, 우주 여행에 대한 준비가 부족하다는 신호로 해석될 수 있다. 이러한 요소들이 결합되어 예측 결과가 "Not Transported"로 나타난다.


---
## 8. OpenAI Function Calling 통합

Enable Agent를 OpenAI의 Function Calling과 통합하여 자연어로 예측을 수행할 수 있다.

In [14]:
tool_definition = agent.generate_tool_definition()

print("=== Function Calling 도구 정의 ===")
print(json.dumps(tool_definition, indent=2, ensure_ascii=False))

=== Function Calling 도구 정의 ===
{
  "type": "function",
  "function": {
    "name": "predict_spaceship_transport",
    "description": "Spaceship Titanic 승객의 다른 차원 이동 여부를 예측하는 딥러닝 에이전트",
    "parameters": {
      "type": "object",
      "properties": {
        "HomePlanet": {
          "type": "integer",
          "description": "출발 행성 (0: Earth, 1: Europa, 2: Mars, 3: Unknown)"
        },
        "CryoSleep": {
          "type": "integer",
          "description": "냉동 수면 여부 (0: No, 1: Yes)"
        },
        "Destination": {
          "type": "integer",
          "description": "목적지 (0: TRAPPIST-1e, 1: PSO J318.5-22, 2: 55 Cancri e)"
        },
        "Age": {
          "type": "number",
          "description": "나이"
        },
        "VIP": {
          "type": "integer",
          "description": "VIP 서비스 여부 (0: No, 1: Yes)"
        },
        "RoomService": {
          "type": "number",
          "description": "룸서비스 지출액"
        },
        "FoodCourt": {
          "type": "number",

In [15]:
def chat_with_agent(user_message: str) -> str:
    """자연어로 에이전트와 대화한다"""
    client = OpenAI()
    tools = [agent.generate_tool_definition()]
    
    messages = [
        {
            "role": "system",
            "content": """당신은 Spaceship Titanic 승객의 다른 차원 이동 여부를 예측하는 AI 어시스턴트다.
사용자가 승객 정보를 제공하면 predict_spaceship_transport 함수를 호출하여 예측을 수행한다.
예측 결과를 친절하게 설명해준다. 문장의 끝은 ~다로 끝낸다."""
        },
        {"role": "user", "content": user_message}
    ]
    
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        tools=tools,
        tool_choice="auto"
    )
    
    response_message = response.choices[0].message
    
    if response_message.tool_calls:
        tool_call = response_message.tool_calls[0]
        function_args = json.loads(tool_call.function.arguments)
        
        prediction_result = agent.predict(function_args)
        
        messages.append({
            "role": "assistant",
            "content": None,
            "tool_calls": [tool_call]
        })
        messages.append({
            "role": "tool",
            "tool_call_id": tool_call.id,
            "name": "predict_spaceship_transport",
            "content": json.dumps(prediction_result, ensure_ascii=False)
        })
        
        final_response = client.chat.completions.create(
            model="gpt-4o",
            messages=messages
        )
        
        return final_response.choices[0].message.content
    
    return response_message.content

In [16]:
user_query = """
지구에서 출발한 25세 승객이 있어요.
냉동 수면 상태이고, 목적지는 55 Cancri e입니다.
VIP는 아니고, 어떤 서비스도 이용하지 않았어요.
이 승객이 다른 차원으로 이동될까요?
"""

print("=== 자연어 질의 ===")
print(user_query)
print()
print("=== AI 응답 ===")
response = chat_with_agent(user_query)
print(response)

=== 자연어 질의 ===

지구에서 출발한 25세 승객이 있어요.
냉동 수면 상태이고, 목적지는 55 Cancri e입니다.
VIP는 아니고, 어떤 서비스도 이용하지 않았어요.
이 승객이 다른 차원으로 이동될까요?


=== AI 응답 ===
제 예측에 따르면, 이 승객은 다른 차원으로 이동될 확률이 약 66.7%다. 승객이 냉동 수면 상태에 있고, VIP가 아니며, 어떤 서비스도 이용하지 않은 점이 이런 예측에 기여했을 가능성이 크다. 따라서 이 승객은 다른 차원으로 이동하게 될 것으로 보인다.


In [17]:
user_query_2 = """
유로파에서 온 50세 VIP 승객입니다.
냉동 수면은 하지 않고 있고, TRAPPIST-1e로 가고 있어요.
룸서비스에 2000, 푸드코트에 3000, 쇼핑몰에 1000, 스파에 5000, VR 데크에 2000을 썼습니다.
이동 예측해주세요.
"""

print("=== 자연어 질의 ===")
print(user_query_2)
print()
print("=== AI 응답 ===")
response_2 = chat_with_agent(user_query_2)
print(response_2)

=== 자연어 질의 ===

유로파에서 온 50세 VIP 승객입니다.
냉동 수면은 하지 않고 있고, TRAPPIST-1e로 가고 있어요.
룸서비스에 2000, 푸드코트에 3000, 쇼핑몰에 1000, 스파에 5000, VR 데크에 2000을 썼습니다.
이동 예측해주세요.


=== AI 응답 ===
제공하신 정보를 바탕으로 예측해본 결과, 이 승객은 다른 차원으로 이동되지 않을 것으로 보인다. 승객은 유로파에서 온 50세의 VIP 승객으로 냉동 수면을 하지 않고 있으며, TRAPPIST-1e로 향하고 있다. 룸서비스, 푸드코트, 쇼핑몰, 스파, 그리고 VR 데크 등을 이용하며 많은 소비를 하고 있는 점을 고려해봤을 때, 이 승객은 일상적인 서비스를 이용하며 여행을 지속할 것으로 판단된다. 예측의 신뢰도는 매우 높아 99.96%로 기록된다. 즐거운 여행이 되길 바란다.
